# Artificial Neural Network

### Importing the libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import re

In [ ]:
tf.__version__

'2.2.0'

## Part 1 - Data Preprocessing

### Importing the dataset

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

##Data Engineering

In [ ]:
def get_title(x):
    name = x['Name']
    title_search = re.search(' ([A-Za-z]+)\.', name)
    if title_search:
        title = title_search.group(1)
    if title in ['Don', 'Major', 'Capt', 'Jonkheer', 'Rev', 'Col','Sir']:
        return 'Mr'
    elif title in ['Countess', 'Mme','Dona','Lady']:
        return 'Mrs'
    elif title in ['Mlle', 'Ms']:
        return 'Miss'
    elif title =='Dr':
        if x['Sex']=='Male':
            return 'Mr'
        else:
            return 'Mrs'
    else:
        return title

In [ ]:
train['Cabin']=train['Cabin'].fillna(0)
test['Cabin']=test['Cabin'].fillna(0)
train['hasCabin']=train['Cabin'].apply(lambda x: 0 if x==0 else 1)
test['hasCabin']=test['Cabin'].apply(lambda x: 0 if x==0 else 1)

In [ ]:
train['FamilyMem']=train.apply(lambda x: x['SibSp']+x['Parch'], axis=1)
test['FamilyMem']=test.apply(lambda x: x['SibSp']+x['Parch'], axis=1)

In [ ]:
train['title']=train.apply(get_title, axis=1)
test['title']=test.apply(get_title, axis=1)

In [ ]:
fcol=['Pclass','Sex','Embarked','hasCabin','title','Age','FamilyMem','Fare']
train_df=train[fcol]
test_df=test[fcol]

In [ ]:
X = train_df.iloc[:, 0:].values
y = train['Survived'].values

##Imputing Missing data

In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan,strategy='mean')
imputer.fit(X[:, 5:])
X[:, 5:] = imputer.transform(X[:, 5:])
imputer2 = SimpleImputer(missing_values=np.nan,strategy='most_frequent')
imputer2.fit(X[:, 0:4])
X[:, 0:4] = imputer2.transform(X[:, 0:4])

### Encoding categorical data

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder',OneHotEncoder(), [0,1,2,3,4])],remainder='passthrough')
X = np.array(ct.fit_transform(X))

### Splitting the dataset into the Training set and Test set

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2, random_state=32)

### Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Part 2 - Building the ANN

### Initializing the ANN

In [ ]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [ ]:
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

### Adding the second hidden layer

In [ ]:
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

### Adding the output layer

In [ ]:
ann.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN

In [ ]:
#For Binary classification, the loss func should be 'binary_crossentropy'
#For Non-Binary classification, the loss func should be 'categorical_crossentropy'
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on the Training set

In [ ]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
23/23 [==============================] - 0s 1ms/step - loss: 0.6355 - accuracy: 0.6292
Epoch 2/100
23/23 [==============================] - 0s 2ms/step - loss: 0.6052 - accuracy: 0.6376
Epoch 3/100
23/23 [==============================] - 0s 1ms/step - loss: 0.5807 - accuracy: 0.6615
Epoch 4/100
23/23 [==============================] - 0s 1ms/step - loss: 0.5602 - accuracy: 0.6868
Epoch 5/100
23/23 [==============================] - 0s 1ms/step - loss: 0.5423 - accuracy: 0.7500
Epoch 6/100
23/23 [==============================] - 0s 1ms/step - loss: 0.5283 - accuracy: 0.7823
Epoch 7/100
23/23 [==============================] - 0s 2ms/step - loss: 0.5142 - accuracy: 0.7865
Epoch 8/100
23/23 [==============================] - 0s 1ms/step - loss: 0.5014 - accuracy: 0.7879
Epoch 9/100
23/23 [==============================] - 0s 1ms/step - loss: 0.4886 - accuracy: 0.8006
Epoch 10/100
23/23 [==============================] - 0s 1ms/step - loss: 0.4778 - accuracy: 0.8034
Epoch 11/

## Part 4 - Making the predictions and evaluating the model

### Predicting the Test set results

In [ ]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)

In [ ]:
print(y_pred)

### Making the Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[97 11]
 [27 44]]


0.7877094972067039